In [1]:

from ragas.testset.generator import TestsetGenerator
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from data import documents, distributions
import nest_asyncio
nest_asyncio.apply()

/Users/richardlai/miniconda3/envs/AIE4-midterm2/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/richardlai/miniconda3/envs/AIE4-midterm2/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/richardlai/miniconda3/envs/AIE4-midterm2/lib/python3.11/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Py

Adding file**** /Users/richardlai/Documents/MyProjects/Education/AI-Makerspace-Cohort-4/midterm/ai-policy-rag/src/vectorstore/pdfs/Blueprint-for-an-AI-Bill-of-Rights.pdf
Adding file**** /Users/richardlai/Documents/MyProjects/Education/AI-Makerspace-Cohort-4/midterm/ai-policy-rag/src/vectorstore/pdfs/NIST.AI.600-1.pdf


### Let's test this fine tuned embedding model.

In [2]:
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [3]:
testset = generator.generate_with_langchain_docs(documents, 20, distributions, with_debugging_logs=True)

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

: 